In [2]:
# =============================================================================
# GLM NEGATIVE BINOMIAL 
# =============================================================================

import os
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import box
from rasterstats import zonal_stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.genmod.families import NegativeBinomial
# --- Nuevas librerías para el análisis espacial ---
import libpysal.weights
from esda.moran import Moran, Moran_Local

# ────────────────────────────────────────────────────────────────────────────────
# Rutas de entrada y salida (sin cambios)
ruta_geoformas = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Inventario_geoformas_karsticas_Dunita.shp"
ruta_contorno  = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Contorno_Dunita.shp"
ruta_dem       = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\dem_clip.tif"
ruta_slope     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\pendiente.tif"
ruta_twi       = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\Raster\TWI.tif"
ruta_cover     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Cobertura_final.shp"
ruta_drain     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Drenajes_clip_POT_final.shp"
ruta_fault     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\SIG\SHP\Fallas_lineam.shp"
output_dir     = r"C:\Users\esteb\Desktop\GEOS\Maestría\2025-1S\Análisis geoespacial\Propuesta_geoformas\PNG"
os.makedirs(output_dir, exist_ok=True)

# ────────────────────────────────────────────────────────────────────────────────
# PASO 1: CARGA DE DATOS (sin cambios)
gdf_points  = gpd.read_file(ruta_geoformas)
gdf_contour = gpd.read_file(ruta_contorno)

# ────────────────────────────────────────────────────────────────────────────────
# PASO 2: GENERAR CUADRÍCULA Y CONTEO (sin cambios)
cell_size = 100
minx, miny, maxx, maxy = gdf_contour.total_bounds
xs = np.arange(minx, maxx, cell_size)
ys = np.arange(miny, maxy, cell_size)
polygons = [box(x, y, x+cell_size, y+cell_size) for x in xs for y in ys]
grid = gpd.GeoDataFrame({'geometry': polygons}, crs=gdf_contour.crs)
grid = gpd.clip(grid, gdf_contour)

join = gpd.sjoin(grid, gdf_points, how='left', predicate='intersects')
counts = join.groupby(join.index).size()
grid['count'] = counts.reindex(grid.index).fillna(0).astype(int)

# ────────────────────────────────────────────────────────────────────────────────
# PASO 3: EXTRAER COVARIABLES ZONALES (sin cambios, con una mejora para evitar errores)
grid['dem_mean']   = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_dem,   stats=['mean'], nodata=-9999)]
grid['slope_mean'] = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_slope, stats=['mean'], nodata=-9999)]
grid['twi_mean']   = [s['mean'] if s and s['mean'] is not None else np.nan for s in zonal_stats(grid, ruta_twi,   stats=['mean'], nodata=-9999)]

# ────────────────────────────────────────────────────────────────────────────────
# PASO 4: UNIR COBERTURA Y CALCULAR DISTANCIAS (sin cambios)
cov_gdf  = gpd.read_file(ruta_cover)[['d_N3_COBER','geometry']].to_crs(grid.crs)
centroids= grid.copy(); centroids['geometry'] = centroids.centroid
cov_join = gpd.sjoin(centroids, cov_gdf, how='left', predicate='within')
grid['cover'] = cov_join['d_N3_COBER'].fillna('None')

drains = gpd.read_file(ruta_drain).to_crs(grid.crs)
faults = gpd.read_file(ruta_fault).to_crs(grid.crs)
union_drains = drains.geometry.unary_union
union_faults = faults.geometry.unary_union
grid['dist_drain'] = centroids.geometry.distance(union_drains)
grid['dist_fault'] = centroids.geometry.distance(union_faults)

# ────────────────────────────────────────────────────────────────────────────────
# PASO 5: PREPARAR DataFrame PARA MODELADO (sin cambios)
df = grid[['count','dem_mean','slope_mean','twi_mean','dist_drain','dist_fault','cover']].copy()
df = df.dropna()
y = df['count'].astype(int)
X_vars = df.drop(columns=['count'])
X_vars = pd.get_dummies(X_vars, columns=['cover'], prefix='cov', drop_first=True)
X = sm.add_constant(X_vars.astype(float))
y = y[X.index]

# ────────────────────────────────────────────────────────────────────────────────
# PASO 6: AJUSTE GLM NEGATIVE BINOMIAL (sin cambios)
model = sm.GLM(y, X, family=NegativeBinomial())
results = model.fit()
print("Resumen del Modelo GLM Binomial Negativo:")
print(results.summary())

# ────────────────────────────────────────────────────────────────────────────────

# PASO 7: CREAR GEO DATAFRAME DE RESULTADOS PARA LA SIGUIENTE CELDA
print("\n--- PASO 7: Creando GeoDataFrame final con resultados del modelo... ---")

# Seleccionamos las celdas del grid original que se usaron en el modelo
grid_model = grid.loc[X.index].copy()

# Añadimos los residuos del modelo como una nueva columna
grid_model['resid_dev'] = results.resid_deviance

print("Variable 'grid_model' creada y lista para ser usada en la siguiente celda.")
# Esta línea de verificación te mostrará todas las columnas disponibles:
print("Columnas disponibles en 'grid_model':", grid_model.columns.to_list())

C:\Users\esteb\AppData\Local\Temp\ipykernel_17452\2168116474.py:65: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  union_drains = drains.geometry.unary_union
C:\Users\esteb\AppData\Local\Temp\ipykernel_17452\2168116474.py:66: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  union_faults = faults.geometry.unary_union
C:\Users\esteb\anaconda3\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


Resumen del Modelo GLM Binomial Negativo:
                 Generalized Linear Model Regression Results                  
Dep. Variable:                  count   No. Observations:                 2426
Model:                            GLM   Df Residuals:                     2400
Model Family:        NegativeBinomial   Df Model:                           25
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3929.7
Date:                Tue, 24 Jun 2025   Deviance:                       485.48
Time:                        16:32:08   Pearson chi2:                 1.04e+03
No. Iterations:                     8   Pseudo R-squ. (CS):            0.08029
Covariance Type:            nonrobust                                         
                                                     coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------

In [4]:
# =============================================================================
# SAR CON COBERTURAS SIGNIFICATIVAS (modelo Lag Espacial)
# =============================================================================

import geopandas as gpd
import pandas as pd
import numpy as np
from libpysal.weights import Queen
from spreg import ML_Lag

# ───────────────────────────────────────────────────────────────────────────────
# 1. Variables explicativas
# ───────────────────────────────────────────────────────────────────────────────
# Variables continuas
vars_continuas = ['dem_mean', 'slope_mean', 'twi_mean', 'dist_drain', 'dist_fault']

# Crear dummies solo de las 2 coberturas significativas
df_sar = grid_model.copy()
df_dummies = pd.get_dummies(df_sar['cover'], prefix='cov')

# Verificar nombres exactos de las coberturas
print("Columnas dummies disponibles:", df_dummies.columns.to_list())

# Extraer las dos dummies necesarias
dummies_seleccionadas = [
    'cov_Bosque fragmentado',
    'cov_Mosaico de cultivos y espacios naturales'
]

# Filtrar las que existan realmente en el dataframe
dummies_seleccionadas = [col for col in dummies_seleccionadas if col in df_dummies.columns]

# Añadir dummies al DataFrame original
df_sar = df_sar.join(df_dummies[dummies_seleccionadas])

# Variables explicativas finales
X_vars = vars_continuas + dummies_seleccionadas

# ───────────────────────────────────────────────────────────────────────────────
# 2. Preparar matrices
# ───────────────────────────────────────────────────────────────────────────────
y = df_sar['count'].values.reshape(-1, 1)
X = df_sar[X_vars].astype(float).values

# ───────────────────────────────────────────────────────────────────────────────
# 3. Matriz de pesos espaciales (Queen)
# ───────────────────────────────────────────────────────────────────────────────
w = Queen.from_dataframe(df_sar)
w.transform = 'r'

# ───────────────────────────────────────────────────────────────────────────────
# 4. Ajustar el modelo SAR (Lag Spatial Autoregressive Model)
# ───────────────────────────────────────────────────────────────────────────────
model_sar = ML_Lag(
    y, X, w=w,
    name_y='count',
    name_x=X_vars,
    name_w='Queen'
)

# ───────────────────────────────────────────────────────────────────────────────
# 5. Imprimir resumen del modelo
# ───────────────────────────────────────────────────────────────────────────────
print(model_sar.summary)



Columnas dummies disponibles: ['cov_Bosque abierto', 'cov_Bosque de galería y/o ripario', 'cov_Bosque fragmentado', 'cov_Cereales', 'cov_Cultivos confinados', 'cov_Cultivos permanentes arbustivos', 'cov_Mosaico de cultivos', 'cov_Mosaico de cultivos y espacios naturales', 'cov_Mosaico de pastos y cultivos', 'cov_None', 'cov_Otros cultivos transitorios', 'cov_Pastos arbolados', 'cov_Pastos enmalezados', 'cov_Pastos limpios', 'cov_Plantación forestal', 'cov_Red vial, ferroviaria y terrenos asociados', 'cov_Tejido urbano continuo', 'cov_Tejido urbano discontinuo', 'cov_Tierras desnudas y degradadas', 'cov_Vegetación secundaria o en transición', 'cov_Zonas verdes urbanas']


C:\Users\esteb\AppData\Local\Temp\ipykernel_17452\3705804226.py:48: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = Queen.from_dataframe(df_sar)


REGRESSION RESULTS
------------------

SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG (METHOD = FULL)
-----------------------------------------------------------------
Data set            :     unknown
Weights matrix      :       Queen
Dependent Variable  :       count                Number of Observations:        2426
Mean dependent var  :      1.4596                Number of Variables   :           9
S.D. dependent var  :      1.7774                Degrees of Freedom    :        2417
Pseudo R-squared    :      0.2938
Spatial Pseudo R-squared:  0.1093
Log likelihood      :  -4496.4363
Sigma-square ML     :      2.2615                Akaike info criterion :    9010.873
S.E of regression   :      1.5038                Schwarz criterion     :    9063.019

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
---------------------------------------------------------------